# Airlines dataset

In [28]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [29]:
import os
os.environ['XLA_PYTHON_CLIENT_MEM_FRACTION']='.5'

'export' is not recognized as an internal or external command,
operable program or batch file.


In [30]:
# from jax.config import config
# config.update("jax_enable_x64", True)

## Load Airlines data

In [31]:
import pandas as pd
from pathlib import Path
import os

In [32]:
file_dir = os.path.abspath('')
os.chdir(file_dir)
print(file_dir)

c:\Users\selinederooij\surfdrive\Code\tensorized_kernel_methods\notebooks


In [33]:
data_path = "../data/airline.csv"
df = pd.read_csv(data_path)

FileNotFoundError: [Errno 2] No such file or directory: '../data/airline.csv'

In [ ]:
df.head()

Month  DayofMonth  DayOfWeek  DepTime  ArrTime  AirTime  Distance  \
0      1           3          4     1203     1331      116       810   
1      1           3          4      454      598      314      2283   
2      1           3          4      652      963      175      1521   
3      1           3          4     1013     1172       79       577   
4      1           4          5      818      880       48       239   

   plane_age  ArrDelay  
0         10       -14  
1         10       -22  
2         10       -17  
3         10         2  
4         10        10

In [ ]:
from math import floor
N, d = df.shape
idx = floor(2*N/3)
df_train = df.iloc[:idx,:]
df_test = df.iloc[idx:,:]


In [ ]:
df_train['ArrDelay'] = (df_train['ArrDelay'] - df_train['ArrDelay'].mean()) / df_train['ArrDelay'].std()

/var/folders/w_/_d1xt2v961l669hh9_yf8dnc0000gn/T/ipykernel_54547/3411369841.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train['ArrDelay'] = (df_train['ArrDelay'] - df_train['ArrDelay'].mean()) / df_train['ArrDelay'].std()


In [ ]:
feature_names = ['Month', 'DayofMonth', 'DayOfWeek', 'DepTime', 'ArrTime', 'AirTime', 'Distance', 'plane_age']
x = df_train[feature_names].to_numpy()

label_name = ['ArrDelay']
y = df_train[label_name].to_numpy()

In [ ]:
x.shape

(3952942, 8)

In [ ]:
x = (x - x.min(axis=0)) / (x.max(axis=0) - x.min(axis=0))

In [ ]:
y = (y - y.mean(axis=0)) / y.std(axis=0)

### Data to JAX array

In [ ]:
from jax import numpy as jnp
x = jnp.array(x)
y = jnp.array(y)

In [ ]:
from jax import random
key = random.PRNGKey(42)

## Tensor Kernel Machine

### Polynomial

In [ ]:
from tkm.model import fit
from jax import jit
from tkm.features import polynomial

fit_compiled = jit(fit)

W= fit(key,x,y,M=20,R=2,feature_map=polynomial,numberSweeps=5)

%time W = fit(key,x,y, feature_map=polynomial)
%time W = fit_compiled(key, x, y, feature_map=polynomial)

KeyboardInterrupt: 

In [ ]:
loss

[5887081.5,
 5868684.0,
 5852947.5,
 5786518.5,
 5754037.0,
 5749995.0,
 5741692.5,
 5725896.5,
 5718140.0,
 5708719.0,
 5704233.0,
 5619731.0,
 5542762.5,
 5521153.0,
 5634999.0,
 5672474.5,
 5545400.0,
 5532472.0,
 5531686.0,
 5464710.0,
 5451585.0,
 5438640.5,
 5431995.0,
 5403375.0,
 5401704.5,
 5400475.5,
 5399960.0,
 5397021.5,
 5389956.0,
 5387088.5,
 5378288.0,
 5377906.0,
 5375984.5,
 5375589.0,
 5376113.0,
 5365985.5,
 5356519.5,
 5352387.0,
 5346017.0,
 5355945.0,
 5343421.5,
 5343146.0,
 5345951.0,
 5335261.0,
 5326116.0,
 5321030.5,
 5316943.0,
 5322492.0,
 5313683.5,
 5312603.0,
 5313831.0,
 5306247.0,
 5299004.0,
 5291290.0,
 5291153.5,
 5292189.5,
 5286456.0,
 5285015.0,
 5285916.0,
 5283170.0,
 5281379.5,
 5272585.0,
 5275935.5,
 5273724.0,
 5270499.5,
 5268933.5,
 5269916.0,
 5269837.5,
 5270604.5,
 5262048.0,
 5265740.5,
 5264445.0,
 5260804.5,
 5259120.5,
 5260269.5,
 5261099.0,
 5262292.5,
 5254369.5,
 5257309.0,
 5256725.5]

#### Predict

In [ ]:
from tkm.model import predict
from tkm.metrics import rmse
predict_compiled = jit(predict)



y_hat = predict(x, W, feature_map=polynomial,M=4,R=2)
err = rmse(y.squeeze(), y_hat)
err

# %timeit predict(x, W, feature_map=polynomial)
# %timeit predict_compiled(x,W, feature_map=polynomial)

DeviceArray(2291.404, dtype=float32)

In [ ]:
# from jax import jit
# from tkm.model import predict_vmap
# predict_vmap_compiled = jit(predict_vmap)

# %timeit y_hat = predict_vmap(x, W, feature_map=polynomial)
# %timeit predict_vmap_compiled(x,W)

### Fourier Features

In [ ]:
from tkm.model import TensorizedKernelMachine as TKM
from tkm.features import fourier

model_fourier = TKM(M=40,R=5,features=fourier)

In [ ]:
!export XLA_PYTHON_CLIENT_MEM_FRACTION=.5

In [ ]:
%%time
W = model_fourier.fit(key,x,y)

CPU times: user 49min 25s, sys: 7min 51s, total: 57min 16s
Wall time: 12min 19s


In [ ]:
!export XLA_PYTHON_CLIENT_MEM_FRACTION=.9

In [ ]:
%%time
W = model_fourier.fit(key,x,y)

CPU times: user 49min 14s, sys: 8min 18s, total: 57min 32s
Wall time: 12min


In [ ]:
model_fourier_b10k = TKM(M=40,R=5,features=fourier, batch_size=10000)

In [ ]:
%%time
W = model_fourier_b10k.fit(key,x,y)

CPU times: user 49min 26s, sys: 7min 34s, total: 57min 1s
Wall time: 12min 10s


In [ ]:
model_fourier_r20 = TKM(M=40,R=20,features=fourier)

In [ ]:
%%time
y_hat = model_fourier.predict(x, W)

CPU times: user 17.5 s, sys: 3.04 s, total: 20.5 s
Wall time: 5.34 s


In [ ]:
from tkm.metrics import rmse
rmse = rmse(y.squeeze(), y_hat)
print(rmse)

nan


In [ ]:
W = fit_compiled(key,x,y)

In [ ]:
%timeit fit_compiled(key,x,y).block_until_ready()

1.02 s ± 679 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
from tkm.model import predict_vmap
predict_vmap_poly = partial(predict_vmap, feature_map=fourier, M=40, R=5, lengthscale=x.std(axis=0).mean())
predict_vmap_compiled = jit(predict_vmap_poly)

In [ ]:
from tkm.metrics import rmse

y_hat = predict_vmap_compiled(x,W).block_until_ready()
err = rmse(y.squeeze(), y_hat)
print(err)

2293.5806


In [ ]:
%timeit predict_vmap_compiled(x,W).block_until_ready()

31.2 ms ± 545 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
